In [57]:
pip install nltk pycld2 wordsegment autocorrect

You should consider upgrading via the '/home/jamelly/.virtualenvs/deeplearning/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [58]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
twitter_samples.fileids()
positive_tweets = [t for t in twitter_samples.strings("positive_tweets.json")]
negative_tweets = [t for t in twitter_samples.strings("negative_tweets.json")] 

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/jamelly/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [59]:
print(len(positive_tweets))
print(len(negative_tweets))

5000
5000


In [60]:
train_pos_tweets = positive_tweets[:3500]
test_pos_tweets = positive_tweets[3500:]
train_neg_tweets = negative_tweets[:3500]
test_neg_tweets = negative_tweets[3500:]

In [61]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

3500
1500
3500
1500


In [62]:
print(len(train_pos_tweets + train_neg_tweets))
print(len(test_pos_tweets + test_neg_tweets))

7000
3000


In [63]:
import pycld2 as cld2

def extract_engTweets(tweets):
    en_tweets = []
    for e in tweets:
      details = cld2.detect(e)
      # Extracts only the string with lang name i.e: 'en'
      if details[2][0][1] == 'en': 
        en_tweets.append(e)
    return en_tweets

en_train_pos_tweets = extract_engTweets(train_pos_tweets)
en_train_neg_tweets = extract_engTweets(train_neg_tweets)
en_test_pos_tweets = extract_engTweets(test_pos_tweets)
en_test_neg_tweets = extract_engTweets(test_neg_tweets)
print(len(train_pos_tweets))
print(len(en_train_pos_tweets))
print(len(train_neg_tweets))
print(len(en_train_neg_tweets))
print(len(test_pos_tweets))
print(len(en_test_pos_tweets))
print(len(test_neg_tweets))
print(len(en_test_neg_tweets))


3500
3147
3500
3058
1500
1365
1500
1326


In [64]:
# Remove duplicate
en_train_pos_tweets = list (set(en_train_pos_tweets))
en_train_neg_tweets = list (set(en_train_neg_tweets))
en_test_pos_tweets = list (set(en_test_pos_tweets))
en_test_neg_tweets = list (set(en_test_neg_tweets))
print(len(en_train_pos_tweets))
print(len(en_train_neg_tweets))
print(len(en_test_pos_tweets))
print(len(en_test_neg_tweets))
# To lowercase
en_train_pos_tweets=[x.lower() for x in en_train_pos_tweets]
en_train_neg_tweets=[x.lower() for x in en_train_neg_tweets]
en_test_pos_tweets=[x.lower() for x in en_test_pos_tweets]
en_test_neg_tweets=[x.lower() for x in en_test_neg_tweets]

3147
3058
1365
1326


In [66]:
#pre processing
import re
import wordsegment
import autocorrect
from wordsegment import load, segment
load()
from autocorrect import Speller
spell=Speller()
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")
tknzr = TweetTokenizer()

punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
testTweet1='@PERKSOFNIALLJH RTed! Goood Luck :)'
testTweet2='My legs hurt so bad :))))))))))))))))))'
testTweet3='#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris trouble for being top engaged members in my community this week :)'
testTweet4='@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!'
def pre_process (tweets):
  for tweet in tweets:  
    tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove links that start with HTTP/HTTPS in the tweet
    tweet = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove other url links
    #tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",tweet).split()) #remove '#' and '@'
    tweet = re.sub(r"RT", ' ', tweet, flags=re.MULTILINE) #remove 'RT'
    tweet = re.sub(r"#(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '#'
    tweet = re.sub(r"@(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '@'
    tweet = ''.join([i for i in tweet if not i in punctuations]) #remove ponctuations
    tweet = ' '.join(segment(tweet)) #Remove repeated letters
    tweet = ' '.join([spell (w) for w in tweet.split()]) #Avoiding misspellings and slang words
    tweet = re.sub(r"\d", "", tweet) #remove digits
    tweet = tweet.lower()
    #print(tweet)
    tokens = tknzr.tokenize(tweet) #tokenize
    #tweet = (" ").join([word for word in tokens if not word in stopwords.words()]) # stopwords filter
    tokens = [word for word in tokens if not word in stopwords.words()]
    tweet = (" ").join(stemmer.stem(token) for token in tokens)
    #print(tweet)
pre_process([testTweet1,testTweet2,testTweet3,testTweet4])



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jamelly/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
pre_process(en_train_pos_tweets)
pre_process(en_train_neg_tweets)
pre_process(en_test_pos_tweets)
pre_process(en_test_neg_tweets)

In [ ]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.75, ngram_range=(1,3), max_features=10000)
X_1 = vectorizer.fit_transform(en_train_pos_tweets)
X_2 = vectorizer.fit_transform(en_train_neg_tweets)
X_3 = vectorizer.fit_transform(en_test_pos_tweets)
X_4 = vectorizer.fit_transform(en_test_neg_tweets)

In [ ]:
print(vectorizer.get_feature_names)